In [1]:
import io
from typing import Dict, List
from datetime import datetime, timedelta
from pydantic import BaseModel
from time import sleep

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import sqlalchemy

In [4]:
import berserk
import chess, chess.pgn

In [5]:
from dash import Dash, html, dcc

In [24]:
from __future__ import annotations
from typing import List, Optional
from pydantic import BaseModel

class User(BaseModel):
    name: str
    id: str

class GlobalAnalysis(BaseModel):
    inaccuracy: int
    mistake: int
    blunder: int
    acpl: int

class Player(BaseModel):
    user: User
    rating: int
    ratingDiff: Optional[int] = 0
    analysis: Optional[GlobalAnalysis] = None

class Players(BaseModel):
    white: Player
    black: Player

class Opening(BaseModel):
    eco: str
    name: str
    ply: int
        
class Judgment(BaseModel):
    name: str
    comment: str

class MoveAnalysis(BaseModel):
    mate: Optional[int] = 'nan'
    eval: Optional[int] = 'nan'
    best: Optional[str]
    variation: Optional[str]
    judgment: Optional[Judgment] = ''

class Clock(BaseModel):
    initial: int
    increment: int
    totalTime: int

class GameModel(BaseModel):
    id: str
    rated: bool
    variant: str
    speed: str
    createdAt: datetime
    lastMoveAt: datetime
    status: str
    players: Players
    winner: Optional[str] = 'draw'
    opening: Optional[Opening] = None
    moves: str
    analysis: Optional[List[MoveAnalysis]] = None
    clock: Clock

## Functions

In [25]:
# Read PGN string into python-chess object
def read_pgn(game_str):
    pgn = io.StringIO(game_str)
    game = chess.pgn.read_game(pgn)
    return game

In [26]:
# Read python-chess object, play and display it in them on a board.
def play_game(game):
    board = game.board()
    for move in game.mainline_moves():
        clear_output(wait=True)
        print(board)
        board.push(move)
        sleep(1)

In [27]:
# Function that returns list of position evaluations for entire game.
# Scores are in centipawn and from White perspective
# If value is None it's because engine saw checkmate in #n
def read_evals(game):
    
    eval_lst = []
    for node in game.mainline():
        if node.eval() != None:
            eval_lst.append(node.eval().white().score())

    return eval_lst

def read_moves(game):
    
    move_lst = [str(node.san()) for node in game.mainline()]
    return(move_lst)

def read_game_eval_recursive(game_node):
    
    if game_node.is_end():
        print(game_node.eval())
        return [None]
    else:
        if game_node.eval()!=None:
            return [game_node.eval().white().score()] + read_game_eval(game_node[0])
        else:
            return read_game_eval(game_node[0])   

In [28]:
class lichess_communication:
    
    lichess_id = None
    API_TOKEN = None
    client = None
    get_opening = True
    get_evals = True
    
    games_lst = []
    games_df = None
    
    def __init__(self, user) -> None:
        
        # Initialize token
        with open('conf/token.txt') as f:
            self.API_TOKEN = f.readline()[:-1]
        
        # Initialize lichess client
        self.lichess_id = user
        session = berserk.TokenSession(self.API_TOKEN)
        self.client = berserk.Client(session=session)
    
    def fetch_games_by_dates(self, since: int, until: int) -> List[GamesModel]:
    
        games_gen = self.client.games.export_by_player(self.lichess_id,
                                                       rated=True,
                                                       since=since,
                                                       until=until,
                                                       evals=self.get_evals,
                                                       opening=self.get_opening)
        
        self.games_lst = [GameModel(**game) for game in games_gen] 

        return self.games_lst
    
        #for game in games_gen:
        #    try:
        #        self.games_lst.append(GameModel(**game))
        #    except:
        #        print(game)
        
    def show_games_info(self) -> None:
        
        if self.games_lst == None:
            print('No games have been fetched')
            return False
        else:
            print(f'Fetched {len(self.games_lst)} games')
            print(f'Last game from  : {self.games_lst[0].createdAt}')
            print(f'First game from : {self.games_lst[-1].createdAt}')


    def fill_df(self) -> pd.DataFrame:

        games_dict = dict(id =[],
                          color = [],
                          opponent = [],
                          time_control = [],
                          datetime = [],
                          opening = [],
                          result = [],
                          moves = [],
                          analysis = [],
                          evals = [],
                          mates = [],
                          judgment = [])

        for game in self.games_lst:

            games_dict['id'].append(game.id)

            games_dict['color'].append('white' if (game.players.white==self.lichess_id) else 'black')
            
            if game.players.white.user.id==self.lichess_id:
                games_dict['opponent'].append(game.players.black.user.id)
            else:
                games_dict['opponent'].append(game.players.white.user.id)
            
            games_dict['datetime'].append(game.createdAt)
            
            games_dict['opening'].append(game.opening.name)
            
            if  game.winner=='draw':
                games_dict['result'].append('draw')
            elif game.winner=='white' and game.players.white.user.id==self.lichess_id:
                games_dict['result'].append('win')
            elif game.winner=='black' and game.players.black.user.id==self.lichess_id:
                games_dict['result'].append('win')
            else:
                games_dict['result'].append('loss')

            games_dict['time_control'].append(game.speed)

           
            games_dict['moves'].append(game.moves.split(' '))
            
            games_dict['analysis'].append(True if game.analysis!=None else False)
            
            print(game.analysis)
            games_dict['evals'].append([move_anal.eval for move_anal in game.analysis] 
                                       if game.analysis!=None else None)
            
            games_dict['mates'].append([move_anal.mate for move_anal in game.analysis] 
                                       if game.analysis!=None else None)
            
            games_dict['judgment'].append([move_anal.judgment.name if move_anal.judgment!='' else '' for move_anal in game.analysis] 
                                          if game.analysis!=None else None)

        self.games_df = pd.DataFrame.from_dict(games_dict, orient='columns')
        #self.games_df.set_index('id', inplace=True)
        
        return self.games_df

## Fetch games from Lichess API into DataFrame

In [29]:
lichess_id = 'miguel0f'

last_Xdays = datetime.now()-timedelta(days=20)
since = int(berserk.utils.to_millis(last_Xdays))
until = int(berserk.utils.to_millis(datetime.now()))

lichess_comm = lichess_communication(lichess_id)
lichess_comm.fetch_games_by_dates(since,until)
lichess_comm.fill_df()

[MoveAnalysis(mate='nan', eval=33, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=12, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=19, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=32, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=13, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=13, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=0, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=-13, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=0, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=0, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=0, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=-25, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=-3, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', e

,id,color,opponent,time_control,datetime,opening,result,moves,analysis,evals,mates,judgment
0,btz02tRd,black,riku_g,classical,2022-07-21 19:58:30.434000+00:00,"Scotch Game: Scotch Gambit, London Defense",loss,"[e4, e5, Nf3, Nc6, d4, exd4, Bc4, Bb4+, c3, dx...",True,"[33, 12, 19, 32, 13, 13, 0, -13, 0, 0, 0, -25,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, , , , , , , , , , , , , Inaccuracy, , , Ina..."
1,mDMXMiw3,black,dompart519,classical,2022-07-21 13:08:47.216000+00:00,"Caro-Kann Defense: Advance Variation, Bayonet ...",loss,"[e4, c6, d4, d5, e5, Bf5, g4, Bg6, e6, fxe6, h...",True,"[33, 56, 52, 28, 28, 50, 0, 44, 18, 9, 0, 133,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, , , , , , , , , , , Mistake, , , , , Inaccu..."
2,zCZx0KC7,black,shemon11,classical,2022-07-21 10:32:47.533000+00:00,Scotch Game,win,"[e4, e5, Nf3, Nc6, d4, Bd6, Bc4, f6, dxe5, fxe...",True,"[33, 12, 19, 32, 13, 125, 50, 288, 282, 521, 4...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, , , , , Mistake, Inaccuracy, Blunder, , Mis..."
3,Vzn667SP,black,branovasil,classical,2022-07-20 20:09:19.264000+00:00,"Scotch Game: Scotch Gambit, London Defense",win,"[e4, e5, Nf3, Nc6, d4, exd4, Bc4, Bb4+, c3, dx...",True,"[33, 12, 19, 32, 13, 13, 0, -13, 0, 0, 0, 99, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, , , , , , , , , , , Inaccuracy, , , , Blund..."
4,My7vAPF8,black,branovasil,classical,2022-07-20 19:43:35.991000+00:00,Englund Gambit Complex: Hartlaub-Charlick Gambit,win,"[d4, e5, dxe5, d6, exd6, Bxd6, c3, Nc6, e4, Qe...",True,"[0, 174, 144, 146, 117, 162, 96, 90, 0, 1, -13...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, Blunder, , , , , Inaccuracy, , Inaccuracy, ..."
5,jKKSsOq4,black,aleksanderdaniloff,blitz,2022-07-20 09:46:06.365000+00:00,Elephant Gambit,win,"[e4, e5, Nf3, d5, Nxe5, dxe4, Nc3, Nf6, d3, Bb...",True,"[33, 12, 19, 123, 76, 82, -57, 89, -19, 0, -48...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, , , Inaccuracy, , , Mistake, Mistake, Inacc..."
6,6hxDZllQ,black,oknekim,blitz,2022-07-19 15:55:34.679000+00:00,Elephant Gambit: Paulsen Countergambit,loss,"[e4, e5, Nf3, d5, exd5, e4, Ng1, Qxd5, Nc3, Qe...",True,"[33, 12, 19, 123, 130, 140, 0, 5, 13, 35, 9, 5...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, , , Inaccuracy, , , Mistake, , , , , , , , ..."
7,Bum7UaeP,black,uos,blitz,2022-07-19 11:31:25.303000+00:00,Russian Game: Stafford Gambit,loss,"[e4, e5, Nf3, Nf6, Nxe5, Nc6, Nxc6, dxc6, Nc3,...",False,None,None,None
8,yupZEyr2,black,man05060135,rapid,2022-07-18 23:04:35.034000+00:00,Scotch Game,draw,"[e4, e5, Nf3, Nc6, d4, d5, Nxe5, dxe4, Nxc6, b...",True,"[33, 12, 19, 32, 13, 93, 106, 98, 40, 32, 21, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, , , , , Inaccuracy, , , Inaccuracy, , , Mis..."
9,dyQv6av1,black,man05060135,rapid,2022-07-18 22:43:01.525000+00:00,Elephant Gambit,win,"[e4, e5, Nf3, d5, Nxe5, dxe4, Nxf7, Kxf7, Bc4+...",False,None,None,None


## Save into Database

In [35]:
from sqlalchemy import create_engine, select, MetaData, Table, Column, Integer, String, DateTime, Boolean
from sqlalchemy.orm import declarative_base, Session
from sqlalchemy_utils import ScalarListType

Base = declarative_base()

dtypes = {'id': String,
          'color': String,
          'opponent': String,
          'time_control': String,
          'datetime': DateTime,
          'opening': String,
          'result': String,
          'moves': ScalarListType(),
          'analysis': Boolean,
          'evals': ScalarListType(float),
          'mates': ScalarListType(float),
          'judgment': ScalarListType()
}

class User(Base):
    __tablename__ = 'user_account'

    id = Column(Integer, primary_key=True)
    name = Column(String)

    def __repr__(self):
        return f'<User(name={self.name})>'

class Games(Base):
    __tablename__ = 'games'
    
    id = Column(dtypes['id'], primary_key=True)
    color = Column(dtypes['color'])
    opponent = Column(dtypes['opponent'])
    time_control = Column(dtypes['time_control'])
    datetime = Column(dtypes['datetime'])
    opening = Column(dtypes['opening'])
    result = Column(dtypes['result'])
    moves = Column(dtypes['moves'])
    analysis = Column(dtypes['analysis'])
    evals = Column(dtypes['evals'])
    mates = Column(dtypes['mates'])
    judgment = Column(dtypes['judgment'])
    
    def __repr__(self):
        return f'<Game(id={self.id}, color={self.color}, result={self.result}, opening={self.opening})>'
    
engine = create_engine('sqlite:///:memory:', echo=True, future=True)
Base.metadata.create_all(engine)


session = Session()
    
lichess_comm.games_df.to_sql('games', 
                             engine, 
                             if_exists='append',
                             dtype=dtypes,
                             index=False,
                             index_label='id')

session.commit()


2022-07-22 01:41:04,968 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-22 01:41:04,969 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-07-22 01:41:04,972 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-22 01:41:04,974 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-07-22 01:41:04,977 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-22 01:41:04,979 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("games")
2022-07-22 01:41:04,982 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-22 01:41:04,985 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("games")
2022-07-22 01:41:04,986 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-22 01:41:04,991 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (id)
)


2022-07-22 01:41:04,993 INFO sqlalchemy.engine.Engine [no key 0.00177s] ()
2022-07-22 01:41:04,997 INFO sqlalchemy.engine.Engine 
CREATE TABLE games (
	id VARC

2022-07-22 01:41:05,061 INFO sqlalchemy.engine.Engine COMMIT


In [39]:
stmt = select(Games.evals).where(Games.color == 'black')

with Session(engine) as session:
    
    rows = session.query(Games).count()
    print(f'** Number of rows : {rows} **')
        
    results = session.execute(stmt).fetchall()
    for result in results:
        print(result[0])

2022-07-22 01:42:01,403 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-22 01:42:01,406 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT games.id AS games_id, games.color AS games_color, games.opponent AS games_opponent, games.time_control AS games_time_control, games.datetime AS games_datetime, games.opening AS games_opening, games.result AS games_result, games.moves AS games_moves, games.analysis AS games_analysis, games.evals AS games_evals, games.mates AS games_mates, games.judgment AS games_judgment 
FROM games) AS anon_1
2022-07-22 01:42:01,407 INFO sqlalchemy.engine.Engine [cached since 55.87s ago] ()
** Number of rows : 43 **
2022-07-22 01:42:01,419 INFO sqlalchemy.engine.Engine SELECT games.evals 
FROM games 
WHERE games.color = ?
2022-07-22 01:42:01,426 INFO sqlalchemy.engine.Engine [generated in 0.00758s] ('black',)
[33.0, 12.0, 19.0, 32.0, 13.0, 13.0, 0.0, -13.0, 0.0, 0.0, 0.0, -25.0, -3.0, 89.0, 39.0, 75.0, -30.0, -30.0, -77.0, -84.0, -82.0, -8

In [38]:
for k in range(len(lichess_comm.games_lst)):
    opponent = lichess_comm.games_lst[k].players.black.user.id
    if opponent == 'man05060135':
        print(lichess_comm.games_lst[k])

id='yupZEyr2' rated=True variant='standard' speed='rapid' createdAt=datetime.datetime(2022, 7, 18, 23, 4, 35, 34000, tzinfo=datetime.timezone.utc) lastMoveAt=datetime.datetime(2022, 7, 18, 23, 46, 55, 91000, tzinfo=datetime.timezone.utc) status='draw' players=Players(white=Player(user=User(name='miguel0f', id='miguel0f'), rating=1693, ratingDiff=-1, analysis=GlobalAnalysis(inaccuracy=10, mistake=3, blunder=4, acpl=55)), black=Player(user=User(name='Man05060135', id='man05060135'), rating=1672, ratingDiff=1, analysis=GlobalAnalysis(inaccuracy=4, mistake=4, blunder=6, acpl=57))) winner='draw' opening=Opening(eco='C44', name='Scotch Game', ply=5) moves='e4 e5 Nf3 Nc6 d4 d5 Nxe5 dxe4 Nxc6 bxc6 Bc4 c5 Bb5+ Ke7 d5 a6 Bc6 Rb8 O-O Bb7 Re1 f5 Nc3 Bxc6 dxc6 Qxd1 Rxd1 Nf6 Bf4 h6 Bxc7 Rc8 Bd6+ Kf7 c7 Bxd6 Rxd6 Rxc7 Rxa6 Rb7 b3 Rhb8 Rd1 Ne8 Kf1 Kf8 Rd5 Nc7 Rxf5+ Kg8 Ra5 Ne6 Re5 Nd4 Nxe4 Nxc2 Raxc5 Nd4 f4 Rf8 g3 Rd7 b4 Nf3 Kg2 Nxe5 Rxe5 Rdd8 a4 Rd4 b5 Rxa4 b6 Ra2+ Kh3 Rb8 Re6 Rb2 f5 R2xb6 Re7 Kf8 Rd

In [ ]:
for k in range(len(lichess_comm.games_lst)):
    speed = lichess_comm.games_lst[k].speed
    pref = lichess_comm.games_lst[k].perf
    if speed !=  pref:
        print(speed, pref)

In [ ]:
print(type(lichess_comm.games_lst[0]))
[move.eval for move in lichess_comm.games_lst[0].analysis]

In [ ]:
len(lichess_comm.games_lst)

In [ ]:
games_df['result'].value_counts()

In [ ]:
games_df.loc[games_df['result']=='loss']

In [ ]:
game = read_pgn(games_lst[12])
game_eval = games_df.iloc[12]
play_game(game)

plt.figure()
plt.plot(game_eval)
plt.show()

## Code snippets to look into data returned by API

In [ ]:
for k in range(len(a)):
    if a[k]['players'][color]['user']['name']!='miguel0f':
        print('ahahah')